In [1]:
%%capture
%load_ext autoreload
%autoreload 2
from setup_nb_env import *

from epsampling.utils import load_csv
# pd.set_option('display.float_format', lambda x: '%.3f' % x)
from epsampling.utils import drop_sers_with_nans
from epsampling.utils import date_str_to_int

DATA_DIR = '/work/users/k/4/k4thryn/Repos/EpSampling/data/'
DT = datetime.today().strftime('%Y%m%d-%H%M%S')

### <font color=blue> Death data

In [14]:
### LOAD CSV
timestamp = '20241002-004543'
fpath = os.path.join(DATA_DIR,'processed', f'processed_naive_deaths_{timestamp}.csv')
df_deaths = pd.read_csv(fpath)

### CLEAN
## Drop rows with nans.
df_deaths.dropna(inplace=True)
## Drop samples with negative inc deaths.
df_deaths = df_deaths[df_deaths.True_inc_deaths >= 0]
display(df_deaths)

,Fips,State_fips,Pop,State_pop,Ratio,Date,Proj_inc_deaths,True_cum_deaths,Cum_deaths_tm1,True_inc_deaths,Naive_inc_deaths
1,1001,1,58239,4997675,0.011653,20200620,83.302156,9.0,6.0,3.0,0.970738
2,1001,1,58239,4997675,0.011653,20200627,68.179357,12.0,9.0,3.0,0.794509
3,1001,1,58239,4997675,0.011653,20200704,76.238975,13.0,12.0,1.0,0.888429
4,1001,1,58239,4997675,0.011653,20200711,88.362530,15.0,13.0,2.0,1.029708
5,1001,1,58239,4997675,0.011653,20200718,114.279381,21.0,15.0,6.0,1.331723
...,...,...,...,...,...,...,...,...,...,...,...
312331,56045,56,6891,576641,0.011950,20220409,8.000000,18.0,18.0,0.0,0.095602
312332,56045,56,6891,576641,0.011950,20220416,5.000000,18.0,18.0,0.0,0.059751
312333,56045,56,6891,576641,0.011950,20220423,4.000000,18.0,18.0,0.0,0.047801
312334,56045,56,6891,576641,0.011950,20220430,4.000000,18.0,18.0,0.0,0.047801


### <font color=blue> ACS data

In [15]:
from epsampling.utils import drop_duplicate_cols

timestamp = '20241001-223952'
fpath = os.path.join(DATA_DIR,'processed',f'formatted_acs_results_{timestamp}.csv')
df_acs = pd.read_csv(fpath,index_col='Fips')

## Remove duplicate columns.
df_acs = drop_duplicate_cols(df_acs)
display(df_acs.head())

,OCC_MBSA_ELCAM_L,IND_AFFHM_MQE,IND_PSMAW_M,HU_OCC_OPRgt200,HU_UIS50P,IND_WHT_x7,HU_UIS2049,EDU_PROF,HH_7pP,OCC_MBSA_CES_CM,...,POP_F,POP_AIANNH,POP_AIAN,EDU_HS,POP_A3564,POP_M,POP_16p_EMP_x7,POP_A25p,POP_NH,POP_x2
Fips,,,,,,,,,,,,,,,,,,,,,
2013,0,0,0,0,0,0,0,1,1,2,...,1395,1501,1505,1648,1760,2014,2213,2598,3050,3409
2016,0,6,9,11,12,47,180,14,21,17,...,2256,523,549,2626,2471,2995,3655,3838,4740,5251
2020,1842,2985,428,499,2642,2860,6892,5415,2311,3709,...,142897,19827,21150,94213,107273,149648,144598,193312,264762,292545
2050,32,90,250,239,9,19,150,102,543,31,...,8790,15427,15640,6485,5770,9724,6454,10025,18131,18514
2060,0,0,0,0,0,2,7,1,12,3,...,369,266,282,411,384,480,443,643,740,849


### <font color=blue> Join dfs to verify that they match up.

In [18]:
df = df_deaths.merge(df_acs, on='Fips')
df.reset_index(inplace=True, drop=True)
display(df)

,Fips,State_fips,Pop,State_pop,Ratio,Date,Proj_inc_deaths,True_cum_deaths,Cum_deaths_tm1,True_inc_deaths,...,POP_F,POP_AIANNH,POP_AIAN,EDU_HS,POP_A3564,POP_M,POP_16p_EMP_x7,POP_A25p,POP_NH,POP_x2
0,1001,1,58239,4997675,0.011653,20200620,83.302156,9.0,6.0,3.0,...,30033,98,98,20751,22690,28206,25871,39614,56464,58239
1,1001,1,58239,4997675,0.011653,20200627,68.179357,12.0,9.0,3.0,...,30033,98,98,20751,22690,28206,25871,39614,56464,58239
2,1001,1,58239,4997675,0.011653,20200704,76.238975,13.0,12.0,1.0,...,30033,98,98,20751,22690,28206,25871,39614,56464,58239
3,1001,1,58239,4997675,0.011653,20200711,88.362530,15.0,13.0,2.0,...,30033,98,98,20751,22690,28206,25871,39614,56464,58239
4,1001,1,58239,4997675,0.011653,20200718,114.279381,21.0,15.0,6.0,...,30033,98,98,20751,22690,28206,25871,39614,56464,58239
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306758,56045,56,6891,576641,0.011950,20220409,8.000000,18.0,18.0,0.0,...,3207,88,90,2980,2724,3684,3044,4892,6763,6891
306759,56045,56,6891,576641,0.011950,20220416,5.000000,18.0,18.0,0.0,...,3207,88,90,2980,2724,3684,3044,4892,6763,6891
306760,56045,56,6891,576641,0.011950,20220423,4.000000,18.0,18.0,0.0,...,3207,88,90,2980,2724,3684,3044,4892,6763,6891
306761,56045,56,6891,576641,0.011950,20220430,4.000000,18.0,18.0,0.0,...,3207,88,90,2980,2724,3684,3044,4892,6763,6891


### <font color=blue> Then separate and deduplicate the acs data. Save as two separate dataframes and then join (on fips) on the fly. (This is to save space and time when saving/loading.)

In [19]:
dff_death.columns

Index(['Fips', 'State_fips', 'Pop', 'State_pop', 'Ratio', 'Date',
       'Proj_inc_deaths', 'True_cum_deaths', 'Cum_deaths_tm1',
       'True_inc_deaths', 'Naive_inc_deaths'],
      dtype='object')

In [22]:
# cols_death = ['Fips', 'State_fips', 'Pop', 'State_pop', 'Ratio', 'Date',
#        'Proj_inc_deaths', 'True_cum_deaths', 'Cum_deaths_tm1',
#        'True_inc_deaths', 'Naive_inc_deaths']
cols_acs = ['Fips'] + [x for x in df.columns if x not in df_deaths.columns]

dff_deaths = df[cols_death]
dff_acs = df[cols_acs]
dff_acs.drop_duplicates(inplace=True)

display(dff_deaths, dff_acs)

,Fips,State_fips,Pop,State_pop,Ratio,Date,Proj_inc_deaths,True_cum_deaths,Cum_deaths_tm1,True_inc_deaths,Naive_inc_deaths
0,1001,1,58239,4997675,0.011653,20200620,83.302156,9.0,6.0,3.0,0.970738
1,1001,1,58239,4997675,0.011653,20200627,68.179357,12.0,9.0,3.0,0.794509
2,1001,1,58239,4997675,0.011653,20200704,76.238975,13.0,12.0,1.0,0.888429
3,1001,1,58239,4997675,0.011653,20200711,88.362530,15.0,13.0,2.0,1.029708
4,1001,1,58239,4997675,0.011653,20200718,114.279381,21.0,15.0,6.0,1.331723
...,...,...,...,...,...,...,...,...,...,...,...
306758,56045,56,6891,576641,0.011950,20220409,8.000000,18.0,18.0,0.0,0.095602
306759,56045,56,6891,576641,0.011950,20220416,5.000000,18.0,18.0,0.0,0.059751
306760,56045,56,6891,576641,0.011950,20220423,4.000000,18.0,18.0,0.0,0.047801
306761,56045,56,6891,576641,0.011950,20220430,4.000000,18.0,18.0,0.0,0.047801


,Fips,OCC_MBSA_ELCAM_L,IND_AFFHM_MQE,IND_PSMAW_M,HU_OCC_OPRgt200,HU_UIS50P,IND_WHT_x7,HU_UIS2049,EDU_PROF,HH_7pP,...,POP_F,POP_AIANNH,POP_AIAN,EDU_HS,POP_A3564,POP_M,POP_16p_EMP_x7,POP_A25p,POP_NH,POP_x2
0,1001,70,50,61,56,291,783,287,524,187,...,30033,98,98,20751,22690,28206,25871,39614,56464,58239
96,1003,1185,226,24,129,14074,2255,2511,3059,659,...,116350,1076,1259,79516,89031,110781,104367,161977,216497,227131
194,1005,43,0,0,16,56,105,42,164,84,...,11898,71,77,10289,7823,13361,8561,17995,24083,25259
292,1007,14,184,0,18,86,254,19,93,64,...,10112,21,21,9938,8347,12300,8223,16057,21778,22412
389,1009,163,90,0,9,1,893,132,286,233,...,29354,70,189,23016,22918,29530,24244,40668,53272,58884
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306269,56037,165,3135,0,30,73,561,404,436,164,...,20609,454,506,16349,16838,21850,20884,27816,35626,42459
306368,56039,209,67,0,74,97,175,404,496,0,...,11039,30,30,6148,9875,12280,14952,17659,19835,23319
306467,56041,0,748,0,8,62,116,125,101,98,...,10077,20,52,8432,7709,10437,9688,13233,18533,20514
306566,56043,63,106,0,0,5,70,17,75,94,...,3764,38,71,2992,2958,4004,3907,5423,6651,7768


In [23]:
## SAVE CSV!
fpath = os.path.join(DATA_DIR,'processed','for_modeling',f'df_deaths_{DT}.csv')
dff_death.to_csv(fpath,index=False)
fpath = os.path.join(DATA_DIR,'processed','for_modeling',f'df_acs_{DT}.csv')
dff_acs.to_csv(fpath,index=False)
print(DT)

# ## READ BACK AND CHECK
# dff = pd.read_csv(fpath)
# display(df, dff)
# dff.dtypes

20241003-131323


In [7]:
### ONE-HOT ENCODE State_fips

# from sklearn.preprocessing import OneHotEncoder

# enc = OneHotEncoder(handle_unknown='ignore')

# X = [['Male', 1], ['Female', 3], ['Female', 2]]
# enc.fit(X)
# # enc.categories_

# # enc.transform([['Female', 1], ['Male', 4]]).toarray()
# # enc.inverse_transform([[0, 1, 1, 0, 0], [0, 0, 0, 1, 0]])
# # enc.get_feature_names_out(['gender', 'group'])

# df_fips = df_deaths[['State_fips']]
# df_fips.drop_duplicates(inplace=True) #.reset_index(inplace=True, drop=True)
# df_fips.reset_index(inplace=True, drop=True)
# df_fips.head()

# # fips_enc = enc.fit_transform(df_fips).toarray()
# # fips_enc

# v = df_fips.values[:1]
# fips_enc = enc.fit(df_fips)
# fipss = fips_enc.transform(v).toarray()
# # df_fips['Fips_enc'] = fips_enc